In [1]:
from bpredict import *
import numpy as np
import matplotlib.pyplot as plt
import pickle

# blackscholes

In [10]:
benchmark = '../benchmarks/blackscholes/blackscholes'
args = (1, '../benchmarks/blackscholes/inputs/small.input', '/dev/null')

blackscholes_pred = BasePredictor(record_trace=True)
blackscholes_runner = ExternalRunner(blackscholes_pred, benchmark, args=args, maxinsts=10**8)
blackscholes_runner.run()

In [11]:
if 'blackscholes_runner' and 'blackscholes_pred' in locals():
    print(blackscholes_runner.stats[0].find('sim_insts'))
    with open('blackscholes_trace.pickle', 'wb') as fp:
        pickle.dump(blackscholes_pred.trace, fp)
        blackscholes_trace = blackscholes_pred.trace
else:
    with open('blackscholes_trace.pickle', 'rb') as fp:
        blackscholes_trace = pickle.load(fp)

[Entry(name='sim_insts', values=(100000001,), description='Number of instructions simulated')]


# PicoSAT

In [4]:
benchmark = '../benchmarks/picosat/picosat'
args = ('../benchmarks/picosat/inputs/uuf200-01.cnf', )

picosat_pred = BasePredictor(record_trace=True)
picosat_runner = ExternalRunner(picosat_pred, benchmark, args=args, maxinsts=10**8)
picosat_runner.run()

In [6]:
if 'picosat_runner' and 'picosat_pred' in locals():
    print(picosat_runner.stats[0].find('sim_insts'))
    with open('picosat_trace.pickle', 'wb') as fp:
        pickle.dump(picosat_pred.trace, fp)
        picosat_trace = picosat_pred.trace
else:
    with open('picosat_trace.pickle', 'rb') as fp:
        picosat_trace = pickle.load(fp)

[Entry(name='sim_insts', values=(100000001,), description='Number of instructions simulated')]


# Sjeng

In [2]:
benchmark = '../benchmarks/sjeng/sjeng'
stdin = '\n'.join([
    'sd 8',
    'setboard 3r2k1/3n1ppp/p1p1b3/1nN1p3/2p1P3/Pr2BPP1/1P5P/3RRBK1 b - - 10 20',
    'quit'
])

sjeng_pred = BasePredictor(record_trace=True)
sjeng_runner = ExternalRunner(sjeng_pred, benchmark, stdin=stdin, maxinsts=10**8)
sjeng_runner.run()

In [3]:
if 'sjeng_runner' and 'sjeng_pred' in locals():
    print(sjeng_runner.stats[0].find('sim_insts'))
    with open('sjeng_trace.pickle', 'wb') as fp:
        pickle.dump(sjeng_pred.trace, fp)
        sjeng_trace = sjeng_pred.trace
else:
    with open('sjeng_trace.pickle', 'rb') as fp:
        sjeng_trace = pickle.load(fp)